In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [5]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

age->numerical col ->wont do anything today.
fever->numerical  ->simple imptuer
cough-> ordinal  categorical-> ordinal encoding
gender->nominal categorical -> one hot encoding
city->nominal categorical-> one hot encoding
hascovid-> label encoding (will not do today)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [7]:
X=df.drop(columns='has_covid')
y=df['has_covid']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(80, 5)
(20, 5)
(80,)
(20,)


In [11]:
df['gender'].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [12]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

##  Transformation without using  Column transformer (Difficult way)

#### firstly, add missing values to fever column
#### use simple imputer to fever col 
#### will replace the missing values with mean

In [13]:
X_train['fever']

43     99.0
62    104.0
3      98.0
71    104.0
45     99.0
      ...  
96    101.0
67     99.0
64    104.0
47    104.0
44    102.0
Name: fever, Length: 80, dtype: float64

In [14]:
from sklearn.impute import SimpleImputer
si= SimpleImputer()
X_train_fever= si.fit_transform(X_train[['fever']])
X_test_fever= si.transform(X_test[['fever']])

In [15]:
print(X_train_fever.shape)

(80, 1)


## Applying Ordinal encoding to cough column

In [16]:
from sklearn.preprocessing import OrdinalEncoder 
oe = OrdinalEncoder(categories=[['Mild', 'Strong']]) ## 0 for mild, 1 for strong
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])

## Applying One hot encoding to gender and city column

In [17]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']]).toarray()
X_test_gender_city=ohe.transform(X_test[['gender','city']]).toarray()


In [18]:
print(X_train_gender_city.shape)
## gender has 2 categories, city has 4 categories.

(80, 6)


## Extracting Age  column

In [19]:

# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [20]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 9)

In [21]:
column_names = ['age', 'fever', 'gender_Female', 'gender_Male', 'city_Kolkata', 'city_Bangalore', 'city_Delhi', 'city_Mumbai', 'cough']
X_train_transformed=pd.DataFrame(X_train_transformed,columns=column_names)

In [22]:
X_train_transformed.head()

,age,fever,gender_Female,gender_Male,city_Kolkata,city_Bangalore,city_Delhi,city_Mumbai,cough
0,22.0,99.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,56.0,104.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,31.0,98.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,75.0,104.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,72.0,99.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## Now, using the Column Transformer (The Easy)

```python
# Syntax
ColumnTransformer(
    transformers=[
        ('transformer_name1', transformer1, [list_of_columns1]),
        ('transformer_name2', transformer2, [list_of_columns2]),
        # ...
    ],
    remainder='passthrough'  # or 'drop'
)
```


In [28]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')


In [29]:
transformer.fit_transform(X_train)

array([[ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  31.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.    

In [30]:
transformer.transform(X_test)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  25.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.    